In [1]:
def bob_common(files_nm):
    import tabula
    from tabula import read_pdf
    import pandas as pd
    import numpy as np
    import re
    import operator


    def find_ifsc(branch_details):
        ifsc_obt = ""
        t = branch_details.split(",")          # t[0]
        #print(t[0])
        ifsc_found = dt.loc[operator.and_(operator.and_(dt["Name of the bank"] == "BANK OF BARODA" , dt["Branch Name"].str.contains("{}".format(t[0].strip(" ")))),dt["CENTRE"].str.contains("{}".format(t[1].strip(" "))))]
        if "BARB" in str(ifsc_found["IFSC Code"]):
            index = str(ifsc_found).find("BARB")
            ifsc_obt = str(ifsc_found)[index: index+11]
        elif "Empty" in str(ifsc_found):
            dts = dt.loc[dt["IFSC Code"] == "BARB0{}".format(t[0].strip(" ").replace(" ","")[:6])]
            if len(dts)== 1:
                ifsc_obt = "BARB0{}".format(t[0].strip(" ").replace(" ","")[:6])
            else:
                print("Branch not found in the dataset!!")
        return ifsc_obt


    dt = pd.read_excel('IFSC codes.xls', header=0)

    tables=read_pdf(files_nm, area=(15, 15, 210, 500))
    df1 = pd.DataFrame()
    df1 = pd.concat([c for c in tables]).drop_duplicates()
    df1.reset_index(drop=True, inplace=True)

    acc_n1 = ""; acc_no1 = ""; addr1 = ""; br_details1 = "";
    cifn1 = ""; ifsc1 = ""; micrn1 = "";

    siz = len(df1.columns)

    if siz > 2:
        br_found = False
        cs_found = False

        for count,i in enumerate(df1[df1.columns[0]]):
            try:
                if "branch details" in i.lower():
                    br_found = True
                if ("name" in i.lower()) and br_found:
                    br_details1 = (df1[df1.columns[0]][count]).lower().strip("name").strip(" ").strip(":").strip(" ")
            except AttributeError:
                pass

        for j in range(1, siz):
            for count,i in enumerate(df1[df1.columns[j]]):
                try:
                    if "customer details" in i.lower():
                        cs_found = True
                    if (("account number" in i.lower()) or ("a/c" in i.lower())) and cs_found:
                        acc_no1 = (df1[df1.columns[j]][count]).lower().strip("a/c no").strip(" ").strip(":").strip(" ")
                    if (("ifsc" in i.lower()) or ("ifs code" in i.lower())) and cs_found:
                        ifsc1 = (df1[df1.columns[j]][count]).lower().strip("ifs code").strip(" ").strip(":").strip(" ")
                    if ("name" in i.lower()) and cs_found:
                        acc_n1 = (df1[df1.columns[j]][count]).lower().strip("name").strip(" ").strip(":").strip(" ")
                    if ("address" in i.lower()) and cs_found:
                        addr1 = (df1[df1.columns[j]][count]).lower().strip("address").strip(" ").strip(":").strip(" ")
                    if ("micr" in i.lower()) and cs_found:
                        micrn1 = (df1[df1.columns[j]][count]).lower().strip("micr").strip(" ").strip(":").strip(" ")
                    if ("cif" in i.lower()) and cs_found:
                        cifn1 = (df1[df1.columns[j]][count]).lower().strip("cif").strip(" ").strip(":").strip(" ")
                except AttributeError:
                    pass
    else:
        for count,i in enumerate(df1[df1.columns[0]]):
            try:
                if "account number" in i.lower():
                    acc_no1 = df1[df1.columns[1]][count]
                if ("ifsc" in i.lower()) or ("ifs code" in i.lower()):
                    ifsc1 = df1[df1.columns[1]][count]
                if "branch" in i.lower():
                    br_details1 = df1[df1.columns[1]][count]
                if "account name" in i.lower():
                    acc_n1 = df1[df1.columns[1]][count]
                if "address" in i.lower():
                    addr1 = df1[df1.columns[1]][count]
                if "micr" in i.lower():
                    micrn1 = df1[df1.columns[1]][count]
                if "cif" in i.lower():
                    cifn1 = df1[df1.columns[1]][count]
            except AttributeError:
                pass


    dfr1 = pd.DataFrame([["Account Name", acc_n1]], columns = ["Customer Information", "Data_Extracted"])
    dfr1.loc[len(dfr1)] = ["Account Number", acc_no1]
    if ifsc1 != "":
        dfr1.loc[len(dfr1)] = ["IFS Code", ifsc1]
    else: dfr1.loc[len(dfr1)] = ["IFS Code", find_ifsc(br_details1)]
    dfr1.loc[len(dfr1)] = ["Branch Name", br_details1]
    dfr1.loc[len(dfr1)] = ["Address", addr1]
    dfr1.loc[len(dfr1)] = ["MICR Code", micrn1]
    dfr1.loc[len(dfr1)] = ["CIF Number", cifn1]
    
 
    if find_ifsc(br_details1) != "":
        out = dfr1
    else:
        out = "IFSC Code is not present"
        
        
    return out

In [2]:
bob_common("bob6.pdf")

'pages' argument isn't specified.Will extract only from page 1 by default.


,Customer Information,Data_Extracted
0,Account Name,AASHIRWAD ENTERPRISES
1,Account Number,07940200000742
2,IFS Code,BARB0NAJAFG
3,Branch Name,"NAJAFGARH ROAD, NEW DELHI"
